In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
col_order = ["pt", "e", "eta", "phi"]
reco = pd.read_csv("../reco.csv", index_col=0)[col_order + ["type", "idx_original"]].values
gen = pd.read_csv("../gen.csv", index_col=0)[col_order + ["pid", "type", "idx_original"]].values

In [ ]:
plt.hist(gen[:, 0], bins=np.linspace(0,5,101));
plt.yscale("log")

In [ ]:
plt.hist(gen[:, 1], bins=np.linspace(0,100,101));
plt.yscale("log")

In [ ]:
len(gen), np.sum(gen[:, 0]>0.2)

In [ ]:
fi = np.load("../ev.npz")

In [ ]:
arr = fi["reco_to_gen"]
arr2 = (arr.T / reco[:, 1]).T
arr3 = arr2 > 0.0

In [ ]:
plt.hist(arr[arr!=0], bins=100);
plt.yscale("log")

In [ ]:
plt.hist(arr2[arr2!=0].flatten(), bins=np.linspace(0.8,1.2, 100));

In [ ]:
import matplotlib
plt.figure(figsize=(20,20))
plt.imshow(arr, cmap="Greys", norm=matplotlib.colors.LogNorm(vmin=0.5, vmax=100))

In [ ]:
plt.figure(figsize=(20,20))
plt.imshow(arr>0.0, cmap="Greys")

In [ ]:
plt.hist(arr3.sum(axis=0), bins=np.linspace(0,40,41), histtype="step", lw=2, density=1.0, label="gen");
plt.hist(arr3.sum(axis=1), bins=np.linspace(0,40,41), histtype="step", lw=2, density=1.0, label="reco");
plt.yscale("log")
plt.legend()

In [ ]:
highest_pt_idx = np.argsort(gen[:, 0])[::-1]

In [ ]:
remaining_indices = np.ones(arr.shape[0], dtype=np.int64)
pairs_reco_gen = {}
unreconstructable = []
for igen in highest_pt_idx:
    if gen[igen, 1] < 1:
        continue
    temp = remaining_indices*arr[:, igen]
    best_reco_idx = np.argmax(temp)
    #print(igen, best_reco_idx)
    if best_reco_idx != 0 and temp.sum()>0.0:
        #remaining_indices[best_reco_idx] = 0
        if not (best_reco_idx in pairs_reco_gen):
            pairs_reco_gen[best_reco_idx] = []
        if arr[best_reco_idx, igen] / reco[best_reco_idx, 1] > 0.2:
            pairs_reco_gen[best_reco_idx] += [(igen, arr[best_reco_idx, igen])]
    else:
        unreconstructable += [igen]

pairs_reco_gen_sorted = {}
for k, v in pairs_reco_gen.items():
    v = sorted(v, key=lambda x: x[1], reverse=True)
    pairs_reco_gen_sorted[k] = v

pairs_reco_gen = pairs_reco_gen_sorted

unreconstructable = np.array(unreconstructable)

In [ ]:
len(gen)

In [ ]:
len(unreconstructable)

In [ ]:
pairs_reco_gen

In [ ]:
plt.hist([len(x[1]) for x in pairs_reco_gen.items()], np.linspace(0,10,11));

In [ ]:
ir = 1836
print("rec", ir, ", ".join(["{:.4f}".format(x) for x in reco[ir]]))
for ig, cmp in pairs_reco_gen.get(ir, []):
    print("gen", ig, ", ".join(["{:.4f}".format(x) for x in gen[ig]]), round(cmp, 4))

In [ ]:
import uproot

In [ ]:
fi = uproot.open("../pfntuple.root")

In [ ]:
#fi.get("ana/pftree").keys()

In [ ]:
sh = fi.get("ana/pftree").array(b'simhit_idx_simcluster')[0]
sh_detid = fi.get("ana/pftree").array(b'simhit_detid')[0]
sh_frac = fi.get("ana/pftree").array(b'simhit_frac')[0]

rh = fi.get("ana/pftree").array(b'rechit_idx_cluster')[0]
rh_detid = fi.get("ana/pftree").array(b'rechit_detid')[0]
rh_e = fi.get("ana/pftree").array(b'rechit_e')[0]


rh_d = {}
for i in range(len(rh_detid)):
    k = rh_detid[i]
    if not (k in rh_d):
        rh_d[k] = 0.0
    rh_d[k] += rh_e[i]

In [ ]:
msk0 = np.nonzero(rh == 292)[0]
msk = np.nonzero(sh == 6566)[0]

In [ ]:
ls = []
se = 0
for i in zip(rh_detid[msk0], rh_e[msk0]):
    if i[0] in ls:
        print("dupe")
    ls += [i[0]]
    se += i[1]
    print(i)

In [ ]:
tot = 0.0
for f, d in zip(sh_frac[msk], sh_detid[msk]):
    x = rh_d.get(d, 0.0)*f
    tot += x
    print(f, d, x)

In [ ]:
tot